#### Question 1

What is the edit distance of the best match between pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [25]:
def readGenome(filename):
    genome=''
    with open(filename, 'r') as f:
        for line in f:
            if not line[0] == '>':
                genome += line.rstrip()
    return genome
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [53]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    for i in range(len(y)+1):
        D[0][i] = 0
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
       
    # Edit distance is the value in the bottom right corner of the matrix
    return min(D[-1]) # just change the return from the original dynamic editing formula

In [ ]:
p = 'GCTGATCGATCGTACG'
print(editDistance(p, genome))

3
CPU times: total: 8.36 s
Wall time: 8.42 s


#### Question 2

What is the edit distance of the best match between pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1?  (Don't consider reverse complements.)

In [ ]:
p = 'GATTTACCAGATTGAG'
print(editDistance(p, genome))

2
CPU times: total: 8.31 s
Wall time: 8.31 s


#### Question 3

In a practical, we saw a function for finding the longest exact overlap (suffix/prefix match) between two strings. The function is copied below.

In [ ]:
def overlap(a, b, min_length=30):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

Say we are concerned only with overlaps that (a) are exact matches (no differences allowed), and (b) are at least **k** bases long. To make an overlap graph, we could call **overlap(a, b, min_length=k)** on every possible pair of reads from the dataset. Unfortunately,
that will be very slow!

Consider this: Say we are using k=6, and we have a read a whose length-6 suffix is **GTCCTA**. Say **GTCCTA** does not occur in any other
read in the dataset. In other words, the 6-mer GTCCTA occurs at the end of read **a** and nowhere else. It follows that a's suffix cannot possibly overlap the prefix of any other read by 6 or more characters.

Put another way, if we want to find the overlaps involving a suffix of read a and a prefix of some other read, we can ignore any reads that don't contain the length-k suffix of a. This is good news because it can save us a lot of work!

Here is a suggestion for how to implement this idea. You don't have to do it this way, but this might help you. Let every k-mer in the dataset have an associated Python **set** (1) object [, which starts out empty. We use a Python dictionary to associate each k-mer with its corresponding **set**. (1) For every k-mer in a read, we add the read to the set object corresponding to that k-mer. If our read is GATTA and k=3, we would add **GATTA** to the set objects for **GAT**, **ATT** and **TTA**. We do this for every read so that, at the end, each set contains all reads containing the corresponding k-mer. (2) Now, for each read **a**, we find all overlaps involving a suffix of **a**. To do this, we take a's length-k suffix, find all reads containing that k-mer (obtained from the corresponding **set**) and call **overlap(a, b, min_length=k)** for each.

The most important point is that we do not call **overlap(a, b, min_length=k)** if b does not contain the length-k suffix of a.

Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore that match. Ignore reverse complements.

* Hint 1: Your function should not take much more than 15 seconds to run on this 10,000-read dataset, and maybe much less than
that. (Our solution takes about 3 seconds.) If your function is much slower, there is a problem somewhere.

* Hint 2: **Remember not to overlap a read with itself**. If you do, your answers will be too high.
* Hint 3: You can test your implementation by making up small examples, then checking that (a) your implementation runs quickly, and (b) you get the same answer as if you had simply called **overlap(a, b, min_length=k)** on every pair of reads. We also have provided a couple examples you can check against .

Picture the overlap graph corresponding to the overlaps just calculated. How many edges are in the graph? In other words, how many
distinct pairs of reads overlap?



In [ ]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip()  # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [ ]:
from collections import defaultdict
def overlap_graph(reads, k):
    # Make index
    index = defaultdict(set)
    for read in reads:
        for i in range(len(read) - k + 1):
            index[read[i:i+k]].add(read)

    # Make graph
    graph = defaultdict(set)
    for r in reads:
        for o in index[r[-k:]]:
            if r != o:
                if overlap(r, o, k):
                    graph[r].add(o)

    edges = 0
    for read in graph:
        edges += len(graph[read])
    return(edges, len(graph))

In [ ]:
seqs, quals = readFastq('ERR266411_1.for_asm.fastq')
edges, suffixes = overlap_graph(seqs, 30)
print(edges)

904746


#### Question 4

Picture the overlap graph corresponding to the overlaps computed for the previous question. How many nodes in this graph have at least one outgoing edge?  (In other words, how many reads have a suffix involved in an overlap?)

In [39]:
print(suffixes)

7161
